
# Data Processing
In this notebook we load and process the raw data to develop the final dataset for the IBM-Z Datathon. We make use of three main datasets for a list of all observed geo-effective CMEs from the post-SOHO era between 1996-2024, and three data sets for features and targets:

#### Geo-effective CMEs:
- The [Richardson and Cane list](https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm); a list of near-Earth CMEs from 1996-2010.
- The [George Mason University CME/ICME list](http://solar.gmu.edu/heliophysics/index.php/GMU_CME/ICME_List); a list of geo-effective CMEs from 2007-2017.
- The [NASA CME Scoreboard](https://kauai.ccmc.gsfc.nasa.gov/CMEscoreboard/); a list of geo-effective CMEs from 2013-2024.

#### Features and Targets:
- The [SOHO-LASCO CME Catalogue](https://cdaw.gsfc.nasa.gov/CME_list/); a list of all CMEs observed from 1996-2024 containing information on physical quantities.
- [OMNIWeb Plus data](https://omniweb.gsfc.nasa.gov/); a list of solar wind features associated with CMEs.
- The [SIDC sunspot data](https://www.sidc.be/SILSO/datafiles); daily total sunspot number encoding information about the solar cycle from 1818-2024.


#### Importing libraries:

In [23]:
# For data manipulation
import pandas as pd

#For data visualisation:
import matplotlib.pyplot as plt

#### Reading files into notebook:

In [26]:
# Read the HTML file into a list of DataFrames
# The header is repeated multiple times in the body of the table
# Force the correct header by setting header=0
list_of_df = pd.read_html("RicharsonCane.html", header=0)

# Concatenate the list of DataFrames into a single DataFrame
df = pd.concat(list_of_df)

# Display the first 5 rows of the dataframe
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df.info())

ValueError: No tables found

In [28]:
OMNIWeb FORMAT OF THE SUBSETTED FILE
    
    ITEMS                      FORMAT   
     
 1 YEAR                          I4        
 2 DOY                           I4        
 3 Hour                          I3        
 4 BX, nT (GSE, GSM)             F6.1      
 5 BY, nT (GSE)                  F6.1      
 6 BZ, nT (GSE)                  F6.1      
 7 SW Plasma Temperature, K      F9.0      
 8 SW Proton Density, N/cm^3     F6.1      
 9 SW Plasma Speed, km/s         F6.0      
10 SW Plasma flow long. angle    F6.1      
11 SW Plasma flow lat. angle     F6.1      
12 Alpha/Prot. ratio             F6.3      
13 Flow pressure                 F6.2      
14 Plasma Beta                   F7.2      
15 R (Sunspot No.)               I4     

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)